<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# BSSN Time-Evolution C Code Generation Library

## Author: Zach Etienne

## This module implements a number of helper functions for generating C-code kernels that solve Einstein's equations in the covariant BSSN formalism [described in this NRPy+ tutorial notebook](Tutorial-BSSN_formulation.ipynb)

**Notebook Status:** <font color = red><b> Not yet validated </b></font>

**Validation Notes:** This module has NOT been validated to exhibit convergence to zero of the Hamiltonian constraint violation at the expected order to the exact solution *after a short numerical evolution of the initial data* (see [plots at bottom](#convergence)), and all quantities have been validated against the [original SENR code](https://bitbucket.org/zach_etienne/nrpy).

### NRPy+ modules that generate needed symbolic expressions:
* [BSSN/BSSN_constraints.py](../edit/BSSN/BSSN_constraints.py); [\[**tutorial**\]](Tutorial-BSSN_constraints.ipynb): Hamiltonian constraint in BSSN curvilinear basis/coordinates
* [BSSN/BSSN_RHSs.py](../edit/BSSN/BSSN_RHSs.py); [\[**tutorial**\]](Tutorial-BSSN_time_evolution-BSSN_RHSs.ipynb): Generates the right-hand sides for the BSSN evolution equations in singular, curvilinear coordinates
* [BSSN/BSSN_gauge_RHSs.py](../edit/BSSN/BSSN_gauge_RHSs.py); [\[**tutorial**\]](Tutorial-BSSN_time_evolution-BSSN_gauge_RHSs.ipynb): Generates the right-hand sides for the BSSN gauge evolution equations in singular, curvilinear coordinates
* [BSSN/Enforce_Detgammahat_Constraint.py](../edit/BSSN/Enforce_Detgammahat_Constraint.py); [**tutorial**](Tutorial-BSSN_enforcing_determinant_gammabar_equals_gammahat_constraint.ipynb): Generates symbolic expressions for enforcing the $\det{\bar{\gamma}}=\det{\hat{\gamma}}$ constraint

## Introduction:
Here we use NRPy+ to generate the C source code kernels necessary to generate C functions needed/useful for evolving forward in time the BSSN equations, including:
1. the BSSN RHS expressions for [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) time integration, with arbitrary gauge choice.
1. the BSSN constraints as a check of numerical error

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#importmodules): Import needed Python modules
1. [Step 2](#helperfuncs): Helper Python functions for C code generation
1. [Step 3.a](#bssnrhs): Generate symbolic BSSN RHS expressions
1. [Step 3.b](#bssnrhs_c_code): Register C function `rhs_eval()` for evaluating BSSN RHS expressions
1. [Step 3.c](#ricci): Generate symbolic expressions for 3-Ricci tensor $\bar{R}_{ij}$
1. [Step 3.d](#ricci_c_code): Register C function `Ricci_eval()` for evaluating 3-Ricci tensor $\bar{R}_{ij}$
1. [Step 4.a](#bssnconstraints): Generate symbolic expressions for BSSN Hamiltonian & momentum constraints
1. [Step 4.b](#bssnconstraints_c_code): Register C function `BSSN_constraints()` for evaluating BSSN Hamiltonian & momentum constraints
1. [Step 5](#enforce3metric): Register C function `enforce_detgammahat_constraint` for enforcing the conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint
1. [Step 6](#validation): Confirm above functions are bytecode-identical to those in `BSSN/BSSN_Ccodegen_library.py`
1. [Step 7](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='importmodules'></a>

# Step 1: Import needed Python modules \[Back to [top](#toc)\]
$$\label{importmodules}$$

In [1]:
# RULES FOR ADDING FUNCTIONS TO THIS ROUTINE:
# 1. The function must be runnable from a multiprocessing environment,
#    which means that the function
# 1.a: cannot depend on previous function calls.
# 1.b: cannot create directories (this is not multiproc friendly)


# Step P1: Import needed NRPy+ core modules:
from outputC import lhrh, add_to_Cfunction_dict  # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
from pickling import pickle_NRPy_env   # NRPy+: Pickle/unpickle NRPy+ environment, for parallel codegen
import os, time             # Standard Python modules for multiplatform OS-level functions, benchmarking
import BSSN.BSSN_RHSs as rhs
import BSSN.BSSN_gauge_RHSs as gaugerhs
import loop as lp

<a id='helperfuncs'></a>

# Step 2: Helper Python functions for C code generation \[Back to [top](#toc)\]
$$\label{helperfuncs}$$

* `print_msg_with_timing()` gives the user an idea of what's going on/taking so long. Also outputs timing info.
* `get_loopopts()` sets up options for NRPy+'s `loop` module
* `register_stress_energy_source_terms_return_T4UU()` registers gridfunctions for $T^{\mu\nu}$ if needed and not yet registered.

In [2]:
# print_msg_with_timing() gives the user an idea of what's going on/taking so long. Also outputs timing info.
def print_msg_with_timing(desc, msg="Symbolic", startstop="start", starttime=0.0):
    CoordSystem = par.parval_from_str("reference_metric::CoordSystem")
    elapsed = time.time()-starttime
    if msg == "Symbolic":
        if startstop == "start":
            print("Generating symbolic expressions for " + desc + " (%s coords)..." % CoordSystem)
            return time.time()
        else:
            print("Finished generating symbolic expressions for "+desc+
                  " (%s coords) in %.1f seconds. Next up: C codegen..." % (CoordSystem, elapsed))
    elif msg == "Ccodegen":
        if startstop == "start":
            print("Generating C code for "+desc+" (%s coords)..." % CoordSystem)
            return time.time()
        else:
            print("Finished generating C code for "+desc+" (%s coords) in %.1f seconds." % (CoordSystem, elapsed))


# get_loopopts() sets up options for NRPy+'s loop module
def get_loopopts(points_to_update, enable_SIMD, enable_rfm_precompute, gridsuffix):
    loopopts = points_to_update
    if enable_SIMD:
        loopopts += ",EnableSIMD"
    if enable_rfm_precompute:
        loopopts += ",Enable_rfm_precompute"
    else:
        loopopts += ",Read_xxs"
    if gridsuffix != "":
        loopopts += ","+gridsuffix.replace("_", "")
    return loopopts


# register_stress_energy_source_terms_return_T4UU() registers gridfunctions
#        for T4UU if needed and not yet registered.
def register_stress_energy_source_terms_return_T4UU(enable_stress_energy_source_terms):
    if enable_stress_energy_source_terms:
        registered_already = False
        for i in range(len(gri.glb_gridfcs_list)):
            if gri.glb_gridfcs_list[i].name == "T4UU00":
                registered_already = True
        if not registered_already:
            return ixp.register_gridfunctions_for_single_rank2("AUXEVOL", "T4UU", "sym01", DIM=4)
        else:
            return ixp.declarerank2("T4UU", "sym01", DIM=4)
    return None

<a id='bssnrhs'></a>

# Step 3.a: Generate symbolic BSSN RHS expressions \[Back to [top](#toc)\]
$$\label{bssnrhs}$$

First we generate the symbolic expressions. Be sure to call this function from within a `reference_metric::enable_rfm_precompute="True"` environment if reference metric precomputation is desired.


`BSSN_RHSs__generate_symbolic_expressions()` supports the following features

* (`"OnePlusLog"` by default) Lapse gauge choice
* (`"GammaDriving2ndOrder_Covariant"` by default) Shift gauge choice
* (disabled by default) Kreiss-Oliger dissipation
* (disabled by default) Stress-energy ($T^{\mu\nu}$) source terms
* (enabled by default) "Leave Ricci symbolic": do not compute the 3-Ricci tensor $\bar{R}_{ij}$ within the BSSN RHSs, which only adds to the extreme complexity of the BSSN RHS expressions. Instead leave computation of $\bar{R}_{ij}$=`RbarDD` to a separate function. Doing this generally increases C-code performance by about 10%.

Two lists are returned by this function:

1. `betaU`: the un-rescaled shift vector $\beta^i$, which is used to perform upwinding.
1. `BSSN_RHSs_SymbExpressions`: the BSSN RHS symbolic expressions, using the `lhrh` named-tuple to store a list of LHSs and RHSs, where each LHS and RHS is defined as follows
    1. LHS = BSSN gridfunction whose time derivative is being computed at grid point `i0,i1,i2`, and 
    1. RHS = time derivative expression for given variable at the given point.

In [3]:
def BSSN_RHSs__generate_symbolic_expressions(LapseCondition="OnePlusLog",
                                             ShiftCondition="GammaDriving2ndOrder_Covariant",
                                             enable_KreissOliger_dissipation=True,
                                             enable_stress_energy_source_terms=False,
                                             leave_Ricci_symbolic=True):
    ######################################
    # START: GENERATE SYMBOLIC EXPRESSIONS
    starttime = print_msg_with_timing("BSSN_RHSs", msg="Symbolic", startstop="start")

    # Returns None if enable_stress_energy_source_terms==False; otherwise returns symb expressions for T4UU
    T4UU = register_stress_energy_source_terms_return_T4UU(enable_stress_energy_source_terms)

    # Evaluate BSSN RHSs:
    import BSSN.BSSN_quantities as Bq
    par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", str(leave_Ricci_symbolic))
    rhs.BSSN_RHSs()

    if enable_stress_energy_source_terms:
        import BSSN.BSSN_stress_energy_source_terms as Bsest
        Bsest.BSSN_source_terms_for_BSSN_RHSs(T4UU)
        rhs.trK_rhs += Bsest.sourceterm_trK_rhs
        for i in range(3):
            # Needed for Gamma-driving shift RHSs:
            rhs.Lambdabar_rhsU[i] += Bsest.sourceterm_Lambdabar_rhsU[i]
            # Needed for BSSN RHSs:
            rhs.lambda_rhsU[i] += Bsest.sourceterm_lambda_rhsU[i]
            for j in range(3):
                rhs.a_rhsDD[i][j] += Bsest.sourceterm_a_rhsDD[i][j]

    par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::LapseEvolutionOption", LapseCondition)
    par.set_parval_from_str("BSSN.BSSN_gauge_RHSs::ShiftEvolutionOption", ShiftCondition)
    gaugerhs.BSSN_gauge_RHSs()  # Can depend on above RHSs
    # Restore BSSN.BSSN_quantities::LeaveRicciSymbolic to False
    par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", "False")

    # Add Kreiss-Oliger dissipation to the BSSN RHSs:
    if enable_KreissOliger_dissipation:
        thismodule = "KO_Dissipation"
        # diss_strength = Bq.cf  # <- another attempt. Maybe should multiply by diss_strength?
        diss_strength = par.Cparameters("REAL", thismodule, "diss_strength", 0.2)*Bq.cf # *Bq.cf*Bq.cf*Bq.cf # cf**1 is found better than cf**4 over the long term.

        alpha_dKOD = ixp.declarerank1("alpha_dKOD")
        cf_dKOD = ixp.declarerank1("cf_dKOD")
        trK_dKOD = ixp.declarerank1("trK_dKOD")
        betU_dKOD = ixp.declarerank2("betU_dKOD", "nosym")
        vetU_dKOD = ixp.declarerank2("vetU_dKOD", "nosym")
        lambdaU_dKOD = ixp.declarerank2("lambdaU_dKOD", "nosym")
        aDD_dKOD = ixp.declarerank3("aDD_dKOD", "sym01")
        hDD_dKOD = ixp.declarerank3("hDD_dKOD", "sym01")
        for k in range(3):
            gaugerhs.alpha_rhs += diss_strength * alpha_dKOD[k] * rfm.ReU[k]  # ReU[k] = 1/scalefactor_orthog_funcform[k]
            rhs.cf_rhs += diss_strength * cf_dKOD[k] * rfm.ReU[k]  # ReU[k] = 1/scalefactor_orthog_funcform[k]
            rhs.trK_rhs += diss_strength * trK_dKOD[k] * rfm.ReU[k]  # ReU[k] = 1/scalefactor_orthog_funcform[k]
            for i in range(3):
                if "2ndOrder" in ShiftCondition:
                    gaugerhs.bet_rhsU[i] += diss_strength * betU_dKOD[i][k] * rfm.ReU[k]  # ReU[k] = 1/scalefactor_orthog_funcform[k]
                gaugerhs.vet_rhsU[i] += diss_strength * vetU_dKOD[i][k] * rfm.ReU[k]  # ReU[k] = 1/scalefactor_orthog_funcform[k]
                rhs.lambda_rhsU[i] += diss_strength * lambdaU_dKOD[i][k] * rfm.ReU[k]  # ReU[k] = 1/scalefactor_orthog_funcform[k]
                for j in range(3):
                    rhs.a_rhsDD[i][j] += diss_strength * aDD_dKOD[i][j][k] * rfm.ReU[k]  # ReU[k] = 1/scalefactor_orthog_funcform[k]
                    rhs.h_rhsDD[i][j] += diss_strength * hDD_dKOD[i][j][k] * rfm.ReU[k]  # ReU[k] = 1/scalefactor_orthog_funcform[k]

    # We use betaU as our upwinding control vector:
    Bq.BSSN_basic_tensors()
    betaU = Bq.betaU

    # END: GENERATE SYMBOLIC EXPRESSIONS
    ######################################

    lhs_names = ["alpha", "cf", "trK"]
    rhs_exprs = [gaugerhs.alpha_rhs, rhs.cf_rhs, rhs.trK_rhs]
    for i in range(3):
        lhs_names.append("betU" + str(i))
        rhs_exprs.append(gaugerhs.bet_rhsU[i])
        lhs_names.append("lambdaU" + str(i))
        rhs_exprs.append(rhs.lambda_rhsU[i])
        lhs_names.append("vetU" + str(i))
        rhs_exprs.append(gaugerhs.vet_rhsU[i])
        for j in range(i, 3):
            lhs_names.append("aDD" + str(i) + str(j))
            rhs_exprs.append(rhs.a_rhsDD[i][j])
            lhs_names.append("hDD" + str(i) + str(j))
            rhs_exprs.append(rhs.h_rhsDD[i][j])

    # Sort the lhss list alphabetically, and rhss to match.
    #   This ensures the RHSs are evaluated in the same order
    #   they're allocated in memory:
    lhs_names, rhs_exprs = [list(x) for x in zip(*sorted(zip(lhs_names, rhs_exprs), key=lambda pair: pair[0]))]

    # Declare the list of lhrh's
    BSSN_RHSs_SymbExpressions = []
    for var in range(len(lhs_names)):
        BSSN_RHSs_SymbExpressions.append(lhrh(lhs=gri.gfaccess("rhs_gfs", lhs_names[var]), rhs=rhs_exprs[var]))

    print_msg_with_timing("BSSN_RHSs", msg="Symbolic", startstop="stop", starttime=starttime)
    return [betaU, BSSN_RHSs_SymbExpressions]

<a id='bssnrhs_c_code'></a>

# Step 3.b: Register C code `rhs_eval()` for BSSN RHS expressions \[Back to [top](#toc)\]
$$\label{bssnrhs_c_code}$$

`add_rhs_eval_to_Cfunction_dict()` supports the following features

* (enabled by default) reference-metric precomputation
* (disabled by default) "golden kernels", which greatly increases the C-code generation time in an attempt to reduce computational cost. Most often this results in no speed-up.
* (enabled by default) SIMD output
* (disabled by default) splitting of RHSs into smaller pieces (multiple loops) to improve performance. Doesn't help much.
* (`"OnePlusLog"` by default) Lapse gauge choice
* (`"GammaDriving2ndOrder_Covariant"` by default) Shift gauge choice
* (disabled by default) enable Kreiss-Oliger dissipation
* (disabled by default) add stress-energy ($T^{\mu\nu}$) source terms
* (enabled by default) "Leave Ricci symbolic": do not compute the 3-Ricci tensor $\bar{R}_{ij}$ within the BSSN RHSs, which only adds to the extreme complexity of the BSSN RHS expressions. Instead leave computation of $\bar{R}_{ij}$=`RbarDD` to a separate function. Doing this generally increases C-code performance by about 10%.

Also to enable parallel C-code kernel generation, the NRPy+ environment is pickled and returned.

In [4]:
def add_rhs_eval_to_Cfunction_dict(includes=None, rel_path_to_Cparams=os.path.join("."),
                                   enable_rfm_precompute=True, enable_golden_kernels=False,
                                   enable_SIMD=True, enable_split_for_optimizations_doesnt_help=False,
                                   LapseCondition="OnePlusLog", ShiftCondition="GammaDriving2ndOrder_Covariant",
                                   enable_KreissOliger_dissipation=False, enable_stress_energy_source_terms=False,
                                   leave_Ricci_symbolic=True):
    gridsuffix = par.parval_from_str("grid::current_gridsuffix")

    if includes is None:
        includes = []
    if enable_SIMD:
        includes += [os.path.join("SIMD", "SIMD_intrinsics.h")]
    FD_functions_enable = bool(par.parval_from_str("finite_difference::FD_functions_enable"))
    if FD_functions_enable:
        includes += ["finite_difference_functions.h"]

    # Set up the C function for the BSSN RHSs
    desc = "Evaluate the BSSN RHSs"
    name = "rhs_eval" + gridsuffix
    params = "const paramstruct *restrict params, "
    if enable_rfm_precompute:
        params += "const rfm_struct" + gridsuffix + " *restrict rfmstruct, "
    params += """
              const REAL *restrict auxevol_gfs,const REAL *restrict in_gfs,REAL *restrict rhs_gfs"""

    # Construct body:
    betaU, BSSN_RHSs_SymbExpressions = \
        BSSN_RHSs__generate_symbolic_expressions(LapseCondition=LapseCondition, ShiftCondition=ShiftCondition,
                                                 enable_KreissOliger_dissipation=enable_KreissOliger_dissipation,
                                                 enable_stress_energy_source_terms=enable_stress_energy_source_terms,
                                                 leave_Ricci_symbolic=leave_Ricci_symbolic)

    starttime = print_msg_with_timing("BSSN_RHSs", msg="Ccodegen", startstop="start")

    FD_outCparams = "outCverbose=False,enable_SIMD=" + str(enable_SIMD)
    FD_outCparams += ",GoldenKernelsEnable=" + str(enable_golden_kernels)
    if gridsuffix != "":
        FD_outCparams += ",gridsuffix=" + gridsuffix

    loopopts = get_loopopts("InteriorPoints", enable_SIMD, enable_rfm_precompute, gridsuffix)
    FDorder = par.parval_from_str("finite_difference::FD_CENTDERIVS_ORDER")
    if enable_split_for_optimizations_doesnt_help and FDorder == 6:
        loopopts += ",DisableOpenMP"
        BSSN_RHSs_SymbExpressions_pt1 = []
        BSSN_RHSs_SymbExpressions_pt2 = []
        for lhsrhs in BSSN_RHSs_SymbExpressions:
            if "BETU" in lhsrhs.lhs or "LAMBDAU" in lhsrhs.lhs:
                BSSN_RHSs_SymbExpressions_pt1.append(lhrh(lhs=lhsrhs.lhs, rhs=lhsrhs.rhs))
            else:
                BSSN_RHSs_SymbExpressions_pt2.append(lhrh(lhs=lhsrhs.lhs, rhs=lhsrhs.rhs))
        preloop = """#pragma omp parallel
    {
"""
        preloopbody = fin.FD_outputC("returnstring", BSSN_RHSs_SymbExpressions_pt1,
                                     params=FD_outCparams,
                                     upwindcontrolvec=betaU)
        preloop += "\n#pragma omp for\n" + lp.simple_loop(loopopts, preloopbody)
        preloop += "\n#pragma omp for\n"
        body = fin.FD_outputC("returnstring", BSSN_RHSs_SymbExpressions_pt2,
                              params=FD_outCparams,
                              upwindcontrolvec=betaU)
        postloop = "\n    } // END #pragma omp parallel\n"
    else:
        preloop = ""
        body = fin.FD_outputC("returnstring", BSSN_RHSs_SymbExpressions,
                              params=FD_outCparams,
                              upwindcontrolvec=betaU)
        postloop = ""
    print_msg_with_timing("BSSN_RHSs", msg="Ccodegen", startstop="stop", starttime=starttime)

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        name=name, params=params,
        preloop=preloop, body=body, loopopts=loopopts, postloop=postloop,
        rel_path_to_Cparams=rel_path_to_Cparams)
    return pickle_NRPy_env()

<a id='ricci'></a>

# Step 3.c: Generate symbolic expressions for 3-Ricci tensor $\bar{R}_{ij}$ \[Back to [top](#toc)\]
$$\label{ricci}$$

As described above, we find a roughly 10% speedup by computing the 3-Ricci tensor $\bar{R}_{ij}$ separately from the BSSN RHS equations and storing the 6 independent components in memory. Here we construct the symbolic expressions for all 6 independent components of $\bar{R}_{ij}$ (which is symmetric under interchange of indices).

`Ricci__generate_symbolic_expressions()` does not support any input parameters.

One list is returned by `Ricci__generate_symbolic_expressions()`: `Ricci_SymbExpressions`, which contains a list of expressions for the six independent components of $\bar{R}_{ij}$, using the `lhrh` named-tuple to store a list of LHSs and RHSs, where each LHS and RHS is defined as follows

1. LHS = gridfunction representation of the component of $\bar{R}_{ij}$, computed at grid point i0,i1,i2, and
1. RHS = expression for given component of $\bar{R}_{ij}$.

In [5]:
def Ricci__generate_symbolic_expressions():
    ######################################
    # START: GENERATE SYMBOLIC EXPRESSIONS
    starttime = print_msg_with_timing("3-Ricci tensor", msg="Symbolic", startstop="start")

    # Evaluate 3-Ricci tensor:
    import BSSN.BSSN_quantities as Bq
    par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", "False")

    # We use betaU as our upwinding control vector:
    Bq.BSSN_basic_tensors()

    # Next compute Ricci tensor
    Bq.RicciBar__gammabarDD_dHatD__DGammaUDD__DGammaU()
    # END: GENERATE SYMBOLIC EXPRESSIONS
    ######################################
    # Must register RbarDD as gridfunctions, as we're outputting them to gridfunctions here:
    foundit = False
    for i in range(len(gri.glb_gridfcs_list)):
        if "RbarDD00" in gri.glb_gridfcs_list[i].name:
            foundit = True
    if not foundit:
        ixp.register_gridfunctions_for_single_rank2("AUXEVOL", "RbarDD", "sym01")

    Ricci_SymbExpressions = [lhrh(lhs=gri.gfaccess("auxevol_gfs", "RbarDD00"), rhs=Bq.RbarDD[0][0]),
                             lhrh(lhs=gri.gfaccess("auxevol_gfs", "RbarDD01"), rhs=Bq.RbarDD[0][1]),
                             lhrh(lhs=gri.gfaccess("auxevol_gfs", "RbarDD02"), rhs=Bq.RbarDD[0][2]),
                             lhrh(lhs=gri.gfaccess("auxevol_gfs", "RbarDD11"), rhs=Bq.RbarDD[1][1]),
                             lhrh(lhs=gri.gfaccess("auxevol_gfs", "RbarDD12"), rhs=Bq.RbarDD[1][2]),
                             lhrh(lhs=gri.gfaccess("auxevol_gfs", "RbarDD22"), rhs=Bq.RbarDD[2][2])]
    print_msg_with_timing("3-Ricci tensor", msg="Symbolic", startstop="stop", starttime=starttime)

    return Ricci_SymbExpressions

<a id='ricci_c_code'></a>

# Step 3.d: Register C function `Ricci_eval()` for evaluating 3-Ricci tensor $\bar{R}_{ij}$ \[Back to [top](#toc)\]
$$\label{ricci_c_code}$$

`add_Ricci_eval_to_Cfunction_dict()` supports the following features

* (enabled by default) reference-metric precomputation
* (disabled by default) "golden kernels", which greatly increases the C-code generation time in an attempt to reduce computational cost. Most often this results in no speed-up.
* (enabled by default) SIMD output
* (disabled by default) splitting of RHSs into smaller pieces (multiple loops) to improve performance. Doesn't help much.

Also to enable parallel C-code kernel generation, the NRPy+ environment is pickled and returned.

In [6]:
def add_Ricci_eval_to_Cfunction_dict(includes=None, rel_path_to_Cparams=os.path.join("."),
                                     enable_rfm_precompute=True, enable_golden_kernels=False, enable_SIMD=True,
                                     enable_split_for_optimizations_doesnt_help=False):
    gridsuffix = par.parval_from_str("grid::current_gridsuffix")

    if includes is None:
        includes = []
    if enable_SIMD:
        includes += [os.path.join("SIMD", "SIMD_intrinsics.h")]
    FD_functions_enable = bool(par.parval_from_str("finite_difference::FD_functions_enable"))
    if FD_functions_enable:
        includes += ["finite_difference_functions.h"]

    # Set up the C function for the 3-Ricci tensor
    desc = "Evaluate the 3-Ricci tensor"
    name = "Ricci_eval" + gridsuffix
    params = "const paramstruct *restrict params, "
    if enable_rfm_precompute:
        params += "const rfm_struct" + gridsuffix + " *restrict rfmstruct, "
    params += "const REAL *restrict in_gfs, REAL *restrict auxevol_gfs"

    # Construct body:
    Ricci_SymbExpressions = Ricci__generate_symbolic_expressions()
    FD_outCparams = "outCverbose=False,enable_SIMD=" + str(enable_SIMD)
    FD_outCparams += ",GoldenKernelsEnable=" + str(enable_golden_kernels)
    if gridsuffix != "":
        FD_outCparams += ",gridsuffix="+gridsuffix
    starttime = print_msg_with_timing("3-Ricci tensor", msg="Ccodegen", startstop="start")
    loopopts = get_loopopts("InteriorPoints", enable_SIMD, enable_rfm_precompute, gridsuffix)
    preloop = ""
    FDorder = par.parval_from_str("finite_difference::FD_CENTDERIVS_ORDER")
    if enable_split_for_optimizations_doesnt_help and FDorder >= 8:
        loopopts += ",DisableOpenMP"
        Ricci_SymbExpressions_pt1 = []
        Ricci_SymbExpressions_pt2 = []
        for lhsrhs in Ricci_SymbExpressions:
            if "RBARDD00" in lhsrhs.lhs or "RBARDD11" in lhsrhs.lhs or "RBARDD22" in lhsrhs.lhs:
                Ricci_SymbExpressions_pt1.append(lhrh(lhs=lhsrhs.lhs, rhs=lhsrhs.rhs))
            else:
                Ricci_SymbExpressions_pt2.append(lhrh(lhs=lhsrhs.lhs, rhs=lhsrhs.rhs))
        preloop = """#pragma omp parallel
    {
#pragma omp for
"""
        preloopbody = fin.FD_outputC("returnstring", Ricci_SymbExpressions_pt1,
                                     params=FD_outCparams)
        preloop += lp.simple_loop(loopopts, preloopbody)
        preloop += "#pragma omp for\n"
        body = fin.FD_outputC("returnstring", Ricci_SymbExpressions_pt2,
                              params=FD_outCparams)
        postloop = "\n    } // END #pragma omp parallel\n"
    else:
        body = fin.FD_outputC("returnstring", Ricci_SymbExpressions,
                              params=FD_outCparams)
        postloop = ""
    print_msg_with_timing("3-Ricci tensor", msg="Ccodegen", startstop="stop", starttime=starttime)

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        name=name, params=params,
        preloop=preloop, body=body, loopopts=loopopts, postloop=postloop,
        rel_path_to_Cparams=rel_path_to_Cparams)
    return pickle_NRPy_env()

<a id='bssnconstraints'></a>

# Step 4.a: Generate symbolic expressions for BSSN Hamiltonian & momentum constraints \[Back to [top](#toc)\]
$$\label{bssnconstraints}$$

Next output the C code for evaluating the BSSN Hamiltonian and momentum constraints [(**Tutorial**)](Tutorial-BSSN_constraints.ipynb). In the absence of numerical error, these constraints should evaluate to zero. However it does not due to numerical (typically truncation) error.

We will therefore measure the constraint violations to gauge the accuracy of our simulation, and, ultimately determine whether errors are dominated by numerical finite differencing (truncation) error as expected.

`BSSN_constraints__generate_symbolic_expressions()` supports the following features:

* (disabled by default) add stress-energy ($T^{\mu\nu}$) source terms
* (disabled by default) output Hamiltonian constraint only

One list is returned by `BSSN_constraints__generate_symbolic_expressions()`: `BSSN_constraints_SymbExpressions`, which contains a list of expressions for the Hamiltonian and momentum constraints (4 elements total), using the `lhrh` named-tuple to store a list of LHSs and RHSs, where each LHS and RHS is defined as follows

1. LHS = gridfunction representation of the BSSN constraint, computed at grid point i0,i1,i2, and
1. RHS = expression for given BSSN constraint

In [7]:
def BSSN_constraints__generate_symbolic_expressions(enable_stress_energy_source_terms=False, output_H_only=False):
    ######################################
    # START: GENERATE SYMBOLIC EXPRESSIONS
    starttime = print_msg_with_timing("BSSN constraints", msg="Symbolic", startstop="start")

    # Define the Hamiltonian constraint and output the optimized C code.
    par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", "True")
    import BSSN.BSSN_constraints as bssncon

    # Returns None if enable_stress_energy_source_terms==False; otherwise returns symb expressions for T4UU
    T4UU = register_stress_energy_source_terms_return_T4UU(enable_stress_energy_source_terms)

    bssncon.BSSN_constraints(add_T4UUmunu_source_terms=False, output_H_only=output_H_only)  # We'll add them below if desired.
    if enable_stress_energy_source_terms:
        import BSSN.BSSN_stress_energy_source_terms as Bsest
        Bsest.BSSN_source_terms_for_BSSN_constraints(T4UU)
        bssncon.H += Bsest.sourceterm_H
        for i in range(3):
            bssncon.MU[i] += Bsest.sourceterm_MU[i]

    BSSN_constraints_SymbExpressions = [lhrh(lhs=gri.gfaccess("aux_gfs", "H"), rhs=bssncon.H)]
    if not output_H_only:
        BSSN_constraints_SymbExpressions += [lhrh(lhs=gri.gfaccess("aux_gfs", "MU0"), rhs=bssncon.MU[0]),
                                             lhrh(lhs=gri.gfaccess("aux_gfs", "MU1"), rhs=bssncon.MU[1]),
                                             lhrh(lhs=gri.gfaccess("aux_gfs", "MU2"), rhs=bssncon.MU[2])]
    par.set_parval_from_str("BSSN.BSSN_quantities::LeaveRicciSymbolic", "False")
    print_msg_with_timing("BSSN constraints", msg="Symbolic", startstop="stop", starttime=starttime)
    # END: GENERATE SYMBOLIC EXPRESSIONS
    ######################################
    return BSSN_constraints_SymbExpressions


<a id='bssnconstraints_c_code'></a>

# Step 4.b: Register C function `BSSN_constraints()` for evaluating BSSN Hamiltonian & momentum constraints \[Back to [top](#toc)\]
$$\label{bssnconstraints_c_code}$$

`add_BSSN_constraints_to_Cfunction_dict()` supports the following features

* (enabled by default) reference-metric precomputation
* (disabled by default) "golden kernels", which greatly increases the C-code generation time in an attempt to reduce computational cost. Most often this results in no speed-up.
* (enabled by default) SIMD output
* (disabled by default) splitting of RHSs into smaller pieces (multiple loops) to improve performance. Doesn't help much.
* (disabled by default) add stress-energy ($T^{\mu\nu}$) source terms
* (disabled by default) output Hamiltonian constraint only

Also to enable parallel C-code kernel generation, the NRPy+ environment is pickled and returned.

In [8]:
def add_BSSN_constraints_to_Cfunction_dict(includes=None, rel_path_to_Cparams=os.path.join("."),
                                           enable_rfm_precompute=True, enable_golden_kernels=False, enable_SIMD=True,
                                           enable_stress_energy_source_terms=False,
                                           output_H_only=False):

    gridsuffix = par.parval_from_str("grid::current_gridsuffix")

    if includes is None:
        includes = []
    if enable_SIMD:
        includes += [os.path.join("SIMD", "SIMD_intrinsics.h")]
    FD_functions_enable = bool(par.parval_from_str("finite_difference::FD_functions_enable"))
    if FD_functions_enable:
        includes += ["finite_difference_functions.h"]

    # Set up the C function for the BSSN constraints
    desc = "Evaluate the BSSN constraints"
    name = "BSSN_constraints" + gridsuffix
    params = "const paramstruct *restrict params, "
    if enable_rfm_precompute:
        params += "const rfm_struct" + gridsuffix + " *restrict rfmstruct, "
    params += """
                 const REAL *restrict in_gfs, const REAL *restrict auxevol_gfs, REAL *restrict aux_gfs"""

    # Construct body:
    BSSN_constraints_SymbExpressions = BSSN_constraints__generate_symbolic_expressions(enable_stress_energy_source_terms,
                                                                                       output_H_only=output_H_only)

    FD_outCparams = "outCverbose=False,enable_SIMD=" + str(enable_SIMD)
    FD_outCparams += ",GoldenKernelsEnable=" + str(enable_golden_kernels)
    if gridsuffix != "":
        FD_outCparams += ",gridsuffix="+gridsuffix
    starttime = print_msg_with_timing("BSSN constraints", msg="Ccodegen", startstop="start")
    body = fin.FD_outputC("returnstring", BSSN_constraints_SymbExpressions,
                          params=FD_outCparams)
    print_msg_with_timing("BSSN constraints", msg="Ccodegen", startstop="stop", starttime=starttime)

    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        name=name, params=params,
        body=body,
        loopopts=get_loopopts("InteriorPoints", enable_SIMD, enable_rfm_precompute, gridsuffix),
        rel_path_to_Cparams=rel_path_to_Cparams)
    return pickle_NRPy_env()

<a id='enforce3metric'></a>

# Step 5: Register C function `enforce_detgammahat_constraint` for enforcing the conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint \[Back to [top](#toc)\]
$$\label{enforce3metric}$$

Then enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (Eq. 53 of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)), as [documented in the corresponding NRPy+ tutorial notebook](Tutorial-BSSN_enforcing_determinant_gammabar_equals_gammahat_constraint.ipynb)

Applying curvilinear boundary conditions should affect the initial data at the outer boundary, and will in general cause the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint to be violated there. Thus after we apply these boundary conditions, we must always call the routine for enforcing the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint.

`add_enforce_detgammahat_constraint_to_Cfunction_dict()` supports the following features

* (enabled by default) reference-metric precomputation
* (disabled by default) "golden kernels", which greatly increases the C-code generation time in an attempt to reduce computational cost. Most often this results in no speed-up.

Also to enable parallel C-code kernel generation, the NRPy+ environment is pickled and returned.

In [9]:
def add_enforce_detgammahat_constraint_to_Cfunction_dict(includes=None, rel_path_to_Cparams=os.path.join("."),
                                                         enable_rfm_precompute=True, enable_golden_kernels=False):
    # This function disables SIMD, as it includes cbrt() and abs() functions.
    gridsuffix = par.parval_from_str("grid::current_gridsuffix")

    if includes is None:
        includes = []
    FD_functions_enable = bool(par.parval_from_str("finite_difference::FD_functions_enable"))
    if FD_functions_enable:
        includes += ["finite_difference_functions.h"]

    # Set up the C function for enforcing the det(gammabar) = det(gammahat) BSSN algebraic constraint
    desc = "Enforce the det(gammabar) = det(gammahat) (algebraic) constraint"
    name = "enforce_detgammahat_constraint" + gridsuffix
    params = "const paramstruct *restrict params, "
    if enable_rfm_precompute:
        params += "const rfm_struct" + gridsuffix + " *restrict rfmstruct, "
    params += "REAL *restrict in_gfs"

    # Construct body:
    enforce_detg_constraint_symb_expressions = EGC.Enforce_Detgammahat_Constraint_symb_expressions()

    FD_outCparams = "outCverbose=False,enable_SIMD=False"
    FD_outCparams += ",GoldenKernelsEnable=" + str(enable_golden_kernels)
    if gridsuffix != "":
        FD_outCparams += ",gridsuffix="+gridsuffix
    starttime = print_msg_with_timing("Enforcing det(gammabar)=det(gammahat) constraint", msg="Ccodegen", startstop="start")
    body = fin.FD_outputC("returnstring", enforce_detg_constraint_symb_expressions,
                          params=FD_outCparams)
    print_msg_with_timing("Enforcing det(gammabar)=det(gammahat) constraint", msg="Ccodegen", startstop="stop", starttime=starttime)

    enable_SIMD = False
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        name=name, params=params,
        body=body,
        loopopts=get_loopopts("AllPoints", enable_SIMD, enable_rfm_precompute, gridsuffix),
        rel_path_to_Cparams=rel_path_to_Cparams)
    return pickle_NRPy_env()

<a id='validation'></a>

# Step 6: Confirm above functions are bytecode-identical to those in `BSSN/BSSN_Ccodegen_library.py` \[Back to [top](#toc)\]
$$\label{validation}$$

In [10]:
import BSSN.BSSN_Ccodegen_library as BCL
import sys

funclist = [("print_msg_with_timing", print_msg_with_timing, BCL.print_msg_with_timing),
            ("get_loopopts", get_loopopts, BCL.get_loopopts),
            ("register_stress_energy_source_terms_return_T4UU", register_stress_energy_source_terms_return_T4UU, BCL.register_stress_energy_source_terms_return_T4UU),
            ("BSSN_RHSs__generate_symbolic_expressions", BSSN_RHSs__generate_symbolic_expressions, BCL.BSSN_RHSs__generate_symbolic_expressions),
            ("add_rhs_eval_to_Cfunction_dict", add_rhs_eval_to_Cfunction_dict, BCL.add_rhs_eval_to_Cfunction_dict),
            ("Ricci__generate_symbolic_expressions", Ricci__generate_symbolic_expressions, BCL.Ricci__generate_symbolic_expressions),
            ("add_Ricci_eval_to_Cfunction_dict", add_Ricci_eval_to_Cfunction_dict, BCL.add_Ricci_eval_to_Cfunction_dict),
            ("BSSN_constraints__generate_symbolic_expressions", BSSN_constraints__generate_symbolic_expressions, BCL.BSSN_constraints__generate_symbolic_expressions),
            ("add_BSSN_constraints_to_Cfunction_dict", add_BSSN_constraints_to_Cfunction_dict, BCL.add_BSSN_constraints_to_Cfunction_dict),
            ("add_enforce_detgammahat_constraint_to_Cfunction_dict", add_enforce_detgammahat_constraint_to_Cfunction_dict, BCL.add_enforce_detgammahat_constraint_to_Cfunction_dict)
           ]

for func in funclist:
    # https://stackoverflow.com/questions/20059011/check-if-two-python-functions-are-equal
    if func[1].__code__.co_code != func[2].__code__.co_code:
        print("ERROR: function " + func[0] + " is not the same as the BSSN.BSSN_Ccodegen_library version!")
        sys.exit(1)

print("PASS! ALL FUNCTIONS ARE BYTECODE IDENTICAL")

PASS! ALL FUNCTIONS ARE BYTECODE IDENTICAL


<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-BSSN_time_evolution-C_codegen_library.pdf](Tutorial-BSSN_time_evolution-C_codegen_library.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [11]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-BSSN_time_evolution-C_codegen_library")

Created Tutorial-BSSN_time_evolution-C_codegen_library.tex, and compiled
    LaTeX file to PDF file Tutorial-BSSN_time_evolution-
    C_codegen_library.pdf
